#Importation des packages:

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

# Recuperation de 5000 hrefs des produits

---



In [ ]:
All_urls_product = []
start = time.time()
for j in range(1,51):
  print('la page',j)
    # Pause avant chaque requete d'un temps arbitraire entre 0.5 et 0.5 s
  time.sleep(random.uniform(0.5,0.5))
  soup = BeautifulSoup(requests.get("https://fr.openfoodfacts.org/"+ str(j)).text, 'html.parser') 
  liste = []
  liste2 = []
  for link in soup.find_all('a'):
      texte = link.get('href')
      liste.append(texte)
  for i in range(0,len(liste)):
      if str(liste[i]).startswith('/produit') is True :
        H = "https://fr.openfoodfacts.org"+liste[i]
        liste2.append(H)
  for o in liste2:
    print(o)
    All_urls_product.append(o)
    
# end time
end = time.time()
# total time taken
print(f"Runtime of the program is {end - start}")

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
https://fr.openfoodfacts.org/produit/3017620429484/nutella-ferrero
https://fr.openfoodfacts.org/produit/7613035989535/le-bon-paris-a-l-etouffee-sans-nitrite-4-tranches-herta
https://fr.openfoodfacts.org/produit/5410188031072/gazpacho-alvalle
https://fr.openfoodfacts.org/produit/5010477348630/muesli-bio-jordans
https://fr.openfoodfacts.org/produit/3033710084913/arome-maggi
https://fr.openfoodfacts.org/produit/3251490332080/biscuits-figue-et-son-gerble
https://fr.openfoodfacts.org/produit/3045140105502/chocolat-au-lait-du-pays-alpin-milka
https://fr.openfoodfacts.org/produit/7613035833272/perrier
https://fr.openfoodfacts.org/produit/8712100325953/amora-moutarde-de-dijon-fine-et-forte-bocal
https://fr.openfoodfacts.org/produit/3168930010906/quaker-cruesli-chocolat-noir
https://fr.openfoodfacts.org/produit/3228857001231/harrys-brioche-tressee-nature-au-sucre-perle-sans-additifs
https://fr.openfoodfacts.org/produi

In [ ]:
len(All_urls_product)

5000

# Recuperation des caractéréstiques de chaque produit
##Creation de la fonction OpenFoodFacts() qui recuepre l'ensemble des informations:

Précision: le scraping de touts les produits de ce site nécissite l'utilisation des cluster de grande puissance de calcul(GPU) dans cloud.


In [ ]:
start = time.time() 

def OpenFoodFacts(url):
    
    OpenFoodFacts = []
    
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')
    
    # recuperation des noms des produits

    try:
        nom_produit = soup.find('h1').text.replace("\xa0","")
    except:
        nom_produit = 'XXX'
    
    # recuperation des codes bares
  
    try:
        Code_bare = soup.find(id='barcode').text  
    except:
        Code_bare = 'XXX'
        
   # recuperation des NutriScores et qualité nutritionnelle
    try:
        Nutri_Score= soup.find('div',{'class':"small-12 xlarge-6 columns"}).find_all('a',{'href':"/nutriscore"})[1].img['alt'].split()[4] 
        if Nutri_Score == 'A':
            j = 'Très bonne'
            qualité_nutritionnelle = j
        elif Nutri_Score == 'B':
            j = 'Bonne'
            qualité_nutritionnelle = j   
        elif Nutri_Score == 'C':
            j = 'moyenne'
            qualité_nutritionnelle = j
        elif Nutri_Score == 'D':
            j = 'Mauvaise'
            qualité_nutritionnelle = j
        elif Nutri_Score == 'E':
            j = 'Mauvaise'
            qualité_nutritionnelle = j

    except:
        Nutri_Score='XXX'
        if Nutri_Score=='XXX':
            qualité_nutritionnelle = 'XXX'

    # recuperation du Nova           

    try:
        Nova = soup.find_all('a',{'href':"/nova"})[1].img['alt'].split()[0] 
    except:
        Nova ='XXX'

     # recuperation d'Eco_score et l'impact environnemental
    try:
        Eco_Score= soup.find_all('a',{'href':"/eco-score-l-impact-environnemental-des-produits-alimentaires"})[1].img['alt'].split()[1] 
        if Eco_Score == 'A':
            j = 'Très faible'
            Impact_environnemental = j
        elif Eco_Score == 'B':
            j = 'faible'
            Impact_environnemental = j    
        elif Eco_Score == 'C':
            j = 'Modéré'
            Impact_environnemental = j
        elif Eco_Score == 'D':
            j = 'Elevé'
            Impact_environnemental = j
        elif Eco_Score == 'E':
            j = 'Très élevé'
            Impact_environnemental = j
    except:
        Eco_Score='XXX'
        if Eco_Score=='XXX':
            Impact_environnemental = 'XXX'
        
  
    Z = soup.find('div',{'class':"medium-12 large-8 xlarge-8 xxlarge-8 columns"}).find_all('p')
    


    Dénomination_Générique = Quantité = Conditionnement = Marques= Catégories= Labels_certifications_récompenses = Origine_des_ingrédients= Lieux_Fabrication= Code_de_tracabilité= Lien_site_fabricant= Magasins = Pays_de_vente = 'XXX'

    
    for i in Z:
        try: 
            if i.span.text.startswith('Dénomination'):
                Dénomination_Générique = i.text.split('\xa0:')[1]
            elif i.span.text.startswith('Quantité'):
                Quantité = i.contents[1]
            elif i.span.text.startswith('Conditionnement'):
                Conditionnement = i.text.split('\xa0:')[1] 
            elif i.span.text.startswith('Marques'):
                Marques = i.text.split('\xa0:')[1]
            elif i.span.text.startswith('Catégories'):
                Catégories = i.text.split('\xa0:')[1]
            elif i.span.text.startswith('Labels, certifications, récompenses'):
                Labels_certifications_récompenses = i.text.split('\xa0:')[1]
            elif i.span.text.startswith('Origine des ingrédients'):
                Origine_des_ingrédients = i.text.split('\xa0:')[1]
            elif i.span.text.startswith('Lieux de fabrication ou de transformation'):
                Lieux_Fabrication = i.text.split('\xa0:')[1]
            elif i.span.text.startswith('Code de traçabilité'):
                Code_de_tracabilité = i.text.split(':')[1]
            elif i.span.text.startswith('Lien vers la page du produit sur le site officiel du fabricant'):
                Lien_site_fabricant = i.text.split('\xa0:')[1]
            elif i.span.text.startswith('Magasins'):
                Magasins = i.text.split(':')[1]
            elif i.span.text.startswith('Pays de vente'):
                Pays_de_vente = i.text.split('\xa0:')[1]
  
        except NameError:
            print ('XXX')
            continue

            # recuperation de l'huile de palme
    Huile = 'XXX'        
    K = soup.find('p',{'id':"ingredients_analysis"})
    try:
        L = K.find_all('span')
        p = L[0]
        p = p.text.strip()
        if p.startswith('H') or p.startswith('S') is True:
          Huile = p
    except:
        Huile = ('XXX')
        
      # recuperation des additifs
    Additif = 'XXX' 
    
    V = soup.find('ul',{'style':"display:block;float:left;"})
    Additif = []
    try:
        L = V.find_all('li')
        if L is not None:
          for i in range(0,len(L)):
                Additif.append(L[i].a.text)
    except:
        Additif.append('XXX')

        # recuperation des matieres grasses,Acides_gras_saturés,Sucre, Sel
    try:    
        Matieres_grasses = soup.findAll("div", {"class":"small-12 xlarge-6 columns"})[1].contents[4]
    except:
        Matieres_grasses ='XXX'
    try:
        Acides_gras_saturés= soup.findAll("div", {"class":"small-12 xlarge-6 columns"})[1].contents[10]
    except:
        Acides_gras_saturés='XXX'
    try:
        Sucre = soup.findAll("div", {"class":"small-12 xlarge-6 columns"})[1].contents[16]
    except:
        Sucre ='XXX'
    try:
        Sel = soup.findAll("div", {"class":"small-12 xlarge-6 columns"})[1].contents[22]
    except:
        Sel ='XXX'
        #Comparaison avec les valeurs moyennes des produits de même catégorie (Catégories cochées)
    try:
        Comparaison = soup.find('label', {"style":"display:inline;font-size:1rem;"}).text.strip() 
    except:
        Comparaison ='XXX'
        

        # recuperation : energieK,energie,Glucides,Score_nutritionnel_France,Protéines dans la table des informations nutritielle.

    energieK = 'XXX'
    energie = 'XXX'
    Glucides = ('XXX')
    Score_nutritionnel_France = ('XXX')
    table1 = soup.find('tr',{'id':"nutriment_energy-kcal_tr"})
    table2 = soup.find('tr',{'id':"nutriment_energy-kj_tr"})
    table3 =  soup.find('tr',{'id':"nutriment_carbohydrates_tr"})
    table4 =  soup.find('tr',{'id':"nutriment_nutrition-score-fr_tr"})
    table5 =  soup.find('tr',{'id':"nutriment_proteins_tr"})
    try:
        energieK = table1.find_all('td')[1].text.strip().split(' ')[0]
        if energieK == '?':
          energieK = ('XXX')
    except:
        energieK = ('XXX')
    try:
      energie = table2.select('td')[1].text.strip().split(' ')[0]
      if energie == '?':
        energie = ('XXX')
    except:
        energie = ('XXX')
    try:
      Glucides = table3.select('td')[1].text.strip().split(' ')[0]
      if Glucides == '?':
        Glucides = ('XXX')
    except:
        Glucides = ('XXX')
        
    try:
        Score_nutritionnel_France = table4.select('td')[1].text.strip().split(' ')[0]
        if Score_nutritionnel_France == '?':
            Score_nutritionnel_France = ('XXX')
    except:
        Score_nutritionnel_France = ('XXX')
        
    try:
        Protéines = table5.select('td')[1].text.strip().split(' ')[0]
        if Protéines == '?':
            Protéines = ('XXX')
    except:
        Protéines = ('XXX')   
    
    timestamp = 'XXX'
    try:
      temps = soup.find('p',{'class':"details"})
      timestamp = temps.time.text.split()[2]
    except:
      timestamp = 'XXX'

    return(nom_produit,Code_bare,Nutri_Score,qualité_nutritionnelle,Nova,Eco_Score,Impact_environnemental,Dénomination_Générique, Quantité,Conditionnement, Marques, Catégories, Labels_certifications_récompenses, Origine_des_ingrédients, Lieux_Fabrication,Code_de_tracabilité, Lien_site_fabricant, Magasins,Pays_de_vente, Huile, Additif,Matieres_grasses, Acides_gras_saturés, Sucre, Sel,Comparaison,energie,energieK,Glucides,Score_nutritionnel_France,Protéines,timestamp)
    

print("Computation time =",  time.time() - start ,'seconde') # calcul de temps de la computation 


Computation time = 0.001112222671508789 seconde


#Recuperation des caracteristiques des 5000 produits:
## A l'aide de la fonction OpenFoodFacts


In [ ]:
start = time.time()
Caractéristiques =[]
for url in range(0,len(All_urls_product)):
    data = OpenFoodFacts(All_urls_product[url])
    Caractéristiques.append(data) 
print("Computation time =",  time.time() - start ,'seconde' )

Computation time = 3323.8710079193115 seconde


## Creation de dataframe


In [ ]:
Dataframe = pd.DataFrame(Caractéristiques ,columns = ['Nom_du_produit','Code_bare','Nutri_score','Qualite_nutritionnelle','Nova','Eco_score','Impact_environnemental','Denomination generique','Quantite','Reconditionnement','Marques','Categories','Labels_certifications_recompenses','Origine_des_ingredients','Lieux_de_fabrication','Code_de_tracabilite','Site_officiel_du_fabricant','Magasins','Pays_de_vente', 'Huile','Additif','Matieres_grasses',' Acides_gras_satures',' Sucre','Sel','Comparaison','Energie','EnergieKcal','Glucides','Score_nutritionnel_France','Proteines','timestamp'])
Dataframe

,Nom_du_produit,Code_bare,Nutri_score,Qualite_nutritionnelle,Nova,Eco_score,Impact_environnemental,Denomination generique,Quantite,Reconditionnement,Marques,Categories,Labels_certifications_recompenses,Origine_des_ingredients,Lieux_de_fabrication,Code_de_tracabilite,Site_officiel_du_fabricant,Magasins,Pays_de_vente,Huile,Additif,Matieres_grasses,Acides_gras_satures,Sucre,Sel,Comparaison,Energie,EnergieKcal,Glucides,Score_nutritionnel_France,Proteines,timestamp
0,"Cristaline Eau de source - 1,5l",3274080005003,A,Très bonne,XXX,XXX,XXX,Eau de source naturelle,"1,5 l","Bouteille, Plastique, Bouchon",Cristaline,"Boissons, Eaux, Eaux de sources, Eaux minéral...","Point Vert, en:Triman, Sans nitrate",France,"France, 72370, Ardenay-sur-Merize","EMB 44068A - Guenrouet (Loire-Atlantique, Fra...",https://www.moneaucristaline.fr/,"Carrefour, Leclerc, Auchan","Côte d'Ivoire, France, Guadeloupe, Italie, Lu...",XXX,[XXX],0 g,0 g,0 g,0.02 g,Eaux minérales naturelles,0,0,0,0,0,2012
1,Nutella - Ferrero - 400g,3017620422003,E,Mauvaise,4,D,Elevé,Pâte à tartiner aux noisettes et au cacao,400 g,"Verre, Couvercle, Plastique, Pot, Opercule, P...","Ferrero, Nutella","Produits à tartiner, Petit-déjeuners, Aides c...","Sans gluten, Point Vert","Australie, Brésil, Chili, Côte d'Ivoire, Ghan...",France,NON COMMUNIQUÉ,https://www.nutella.com/fr/fr/produits/n...,"Bi1, Magasins U, Carrefour, Franprix, Auchan","Algérie, Autriche, Belgique, Canada, France, ...",Huile de palme,[E322 - Lécithines],30.9 g,10.6 g,56.3 g,0.107 g,Aide culinaire sucrée,2 252,539,"57,5",26,"6,3",2016
2,Prince Chocolat - Lu - 300g,7622210449283,D,Mauvaise,4,D,Elevé,BISCUITS FOURRÉS (35%) PARFUM CHOCOLAT,300 g,"Plastique, Film en plastique, Paquet, Mondele...",Lu,"Snacks, Snacks sucrés, Biscuits et gâteaux, B...",Sans conservateurs,XXX,XXX,XXX,https://www.lu.fr/prince,"Carrefour Market, Magasins U, Auchan, Interma...","Algérie, Belgique, France, Polynésie français...",Huile de palme,"[E503ii - Carbonate acide d'ammonium, E500ii -...",17 g,5.6 g,32 g,0.58 g,Biscuits au chocolat,1 955,465,69,15,"6,4",2015
3,Coca-Cola - 330ml,5449000000996,E,Mauvaise,4,XXX,XXX,Soda aux extraits végétaux,330 ml,aluminium-can,Coca-cola,"Boissons, Boissons gazeuses, Sodas, Sodas au ...",en:Verified,XXX,XXX,XXX,http://coca-cola.pl coke.at,"Tesco, Auchan, Carrefour","Algérie, Andorre, Autriche, Belgique, Brésil,...",XXX,"[E290 - Dioxyde de carbone, E150d - Caramel au...",0 g,0 g,10.6 g,0 g,Sodas au cola,180,42,"10,6",14,0,2012
4,Nutella - Ferrero - 1kg,3017620425035,E,Mauvaise,4,D,Elevé,Pâte à tartiner aux noisettes,1 kg,"Pot, Verre, Couvercle, Plastique, Carton, Plaque","Ferrero, Nutella","Produits à tartiner, Petit-déjeuners, Produit...",Point Vert,XXX,XXX,XXX,https://www.nutella.com/fr/fr/produits/n...,Auchan,"Algérie, Belgique, France, Allemagne, Guadelo...",Huile de palme,[E322 - Lécithines],30.9 g,10.6 g,56.3 g,0.107 g,Pâtes à tartiner aux noisettes et au cacao,2 252,539,"57,5",26,"6,3",2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,Jambon Serrano - Dulano - 100g,20087487,D,Mauvaise,3,E,Très élevé,"Jambon cru espagnol, séché",100 g,"Barquette, Plastique, Frais, Barquette",Dulano,"Viandes, Charcuteries, Produits à la viande, ...",STG\n,Espagne,"Allemagne, Espagne",DE NI 33333 EG,XXX,LIDL,"France, Allemagne, Suisse",Sans huile de palme,"[E250 - Nitrite de sodium, E252 - Nitrate de p...",12 g,4.5 g,1 g,30 g,Jambons Serrano,XXX,232,1,16,0,2016
4996,Cracotte saveur briochée - LU - 240g,3017760751995,C,moyenne,4,B,faible,Tartines craquantes goût brioché,240 g,Carton,LU,"Aliments et boissons à base de végétaux, Alim...",XXX,XXX,XXX,XXX,XXX,Magasins U,France,Huile de palme,[E322i - Lécithine],2.6 g,1 g,13 g,1.3 g,Tartines craquantes,1 610,380,77,9,11,2015
4997,Salade à la mexicaine - Petit navire - 220g,3019080021505,A,Très bonne,4,XXX,XXX,Salade de thon à la mexicaine,220 g,Barquette,Petit navire,"Plats préparés, Sala

## Exportation dans sous formats csv

In [ ]:
Dataframe.to_csv('./OpenFoodFacts.csv', index=False, encoding='utf-8')  # Exportation de dataframe dans un csv

In [ ]:
data = pd.read_csv('/content/OpenFoodFacts.csv')
data

,Nom_du_produit,Code_bare,Nutri_score,Qualite_nutritionnelle,Nova,Eco_score,Impact_environnemental,Denomination generique,Quantite,Reconditionnement,Marques,Categories,Labels_certifications_recompenses,Origine_des_ingredients,Lieux_de_fabrication,Code_de_tracabilite,Site_officiel_du_fabricant,Magasins,Pays_de_vente,Huile,Additif,Matieres_grasses,Acides_gras_satures,Sucre,Sel,Comparaison,Energie,EnergieKcal,Glucides,Score_nutritionnel_France,Proteines,timestamp
0,"Cristaline Eau de source - 1,5l",3274080005003,A,Très bonne,XXX,XXX,XXX,Eau de source naturelle,"1,5 l","Bouteille, Plastique, Bouchon",Cristaline,"Boissons, Eaux, Eaux de sources, Eaux minéral...","Point Vert, en:Triman, Sans nitrate",France,"France, 72370, Ardenay-sur-Merize","EMB 44068A - Guenrouet (Loire-Atlantique, Fra...",https://www.moneaucristaline.fr/,"Carrefour, Leclerc, Auchan","Côte d'Ivoire, France, Guadeloupe, Italie, Lu...",XXX,['XXX'],0 g,0 g,0 g,0.02 g,Eaux minérales naturelles,0,0,0,0,0,2012
1,Nutella - Ferrero - 400g,3017620422003,E,Mauvaise,4,D,Elevé,Pâte à tartiner aux noisettes et au cacao,400 g,"Verre, Couvercle, Plastique, Pot, Opercule, P...","Ferrero, Nutella","Produits à tartiner, Petit-déjeuners, Aides c...","Sans gluten, Point Vert","Australie, Brésil, Chili, Côte d'Ivoire, Ghan...",France,NON COMMUNIQUÉ,https://www.nutella.com/fr/fr/produits/n...,"Bi1, Magasins U, Carrefour, Franprix, Auchan","Algérie, Autriche, Belgique, Canada, France, ...",Huile de palme,['E322 - Lécithines'],30.9 g,10.6 g,56.3 g,0.107 g,Aide culinaire sucrée,2 252,539,"57,5",26,"6,3",2016
2,Prince Chocolat - Lu - 300g,7622210449283,D,Mauvaise,4,D,Elevé,BISCUITS FOURRÉS (35%) PARFUM CHOCOLAT,300 g,"Plastique, Film en plastique, Paquet, Mondele...",Lu,"Snacks, Snacks sucrés, Biscuits et gâteaux, B...",Sans conservateurs,XXX,XXX,XXX,https://www.lu.fr/prince,"Carrefour Market, Magasins U, Auchan, Interma...","Algérie, Belgique, France, Polynésie français...",Huile de palme,"[""E503ii - Carbonate acide d'ammonium"", 'E500i...",17 g,5.6 g,32 g,0.58 g,Biscuits au chocolat,1 955,465,69,15,"6,4",2015
3,Coca-Cola - 330ml,5449000000996,E,Mauvaise,4,XXX,XXX,Soda aux extraits végétaux,330 ml,aluminium-can,Coca-cola,"Boissons, Boissons gazeuses, Sodas, Sodas au ...",en:Verified,XXX,XXX,XXX,http://coca-cola.pl coke.at,"Tesco, Auchan, Carrefour","Algérie, Andorre, Autriche, Belgique, Brésil,...",XXX,"['E290 - Dioxyde de carbone', ""E150d - Caramel...",0 g,0 g,10.6 g,0 g,Sodas au cola,180,42,"10,6",14,0,2012
4,Nutella - Ferrero - 1kg,3017620425035,E,Mauvaise,4,D,Elevé,Pâte à tartiner aux noisettes,1 kg,"Pot, Verre, Couvercle, Plastique, Carton, Plaque","Ferrero, Nutella","Produits à tartiner, Petit-déjeuners, Produit...",Point Vert,XXX,XXX,XXX,https://www.nutella.com/fr/fr/produits/n...,Auchan,"Algérie, Belgique, France, Allemagne, Guadelo...",Huile de palme,['E322 - Lécithines'],30.9 g,10.6 g,56.3 g,0.107 g,Pâtes à tartiner aux noisettes et au cacao,2 252,539,"57,5",26,"6,3",2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,Jambon Serrano - Dulano - 100g,20087487,D,Mauvaise,3,E,Très élevé,"Jambon cru espagnol, séché",100 g,"Barquette, Plastique, Frais, Barquette",Dulano,"Viandes, Charcuteries, Produits à la viande, ...",STG\n,Espagne,"Allemagne, Espagne",DE NI 33333 EG,XXX,LIDL,"France, Allemagne, Suisse",Sans huile de palme,"['E250 - Nitrite de sodium', 'E252 - Nitrate d...",12 g,4.5 g,1 g,30 g,Jambons Serrano,XXX,232,1,16,0,2016
4996,Cracotte saveur briochée - LU - 240g,3017760751995,C,moyenne,4,B,faible,Tartines craquantes goût brioché,240 g,Carton,LU,"Aliments et boissons à base de végétaux, Alim...",XXX,XXX,XXX,XXX,XXX,Magasins U,France,Huile de palme,['E322i - Lécithine'],2.6 g,1 g,13 g,1.3 g,Tartines craquantes,1 610,380,77,9,11,2015
4997,Salade à la mexicaine - Petit navire - 220g,3019080021505,A,Très bonne,4,XXX,XXX,Salade de thon à la mexicaine,220 g,Barquette,Petit navire,"Plats pré